In [1]:
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.1 MB/s eta 0:00:00
Enter your Groq API Key: ··········


In [2]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.7
)

router_llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

In [17]:

MODEL_CONFIG = {
    "technical": "You are a technical support expert. Be precise, code-focused, and provide step-by-step debugging help.",
    "billing": "You are a billing support expert. Be empathetic, explain policies clearly, and guide users about payments and refunds.",
    "general": "You are a friendly and helpful assistant."
}

In [18]:
def route_prompt(user_input: str) -> str:
    prompt = f"""
    Classify the following user query into exactly one of these categories:
    [technical, billing, general]

    Return ONLY the category name.

    Query: {user_input}
    """

    response = router_llm.invoke(prompt)
    return response.content.strip().lower()

In [19]:
def process_request(user_input: str) -> str:
    category = route_prompt(user_input)

    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]

    messages = [
        ("system", system_prompt),
        ("human", user_input)
    ]

    response = llm.invoke(messages)

    return f"[ROUTED → {category.upper()} EXPERT]\n\n{response.content}"

In [22]:
test_queries = [
    "My python script is throwing IndexError on line 5",
    "I was charged twice for my subscription this month",
]

for q in test_queries:
    print("User:", q)
    print(process_request(q))
    print("-"*80)

User: My python script is throwing IndexError on line 5
[ROUTED → TECHNICAL EXPERT]

To assist you with the `IndexError` on line 5 of your Python script, I'll need more information. 

1. **Please provide the code**: Share the code snippet that's causing the issue, focusing on the lines around line 5.
2. **Error message details**: Share the full error message you're seeing, as it might give clues about what's going wrong.
3. **Expected behavior**: Describe what you expect the script to do when it runs without errors.

With this information, I can provide a step-by-step debugging guide to help resolve the `IndexError`. 

Here's a general approach to debugging `IndexError`:

### Understanding IndexError
- `IndexError` typically occurs when you try to access an element in a sequence (like a list, tuple, or string) using an index that doesn't exist.
- For example, if you have a list `my_list = [1, 2, 3]`, trying to access `my_list[3]` will raise an `IndexError` because list indices in Pytho

In [23]:
def fetch_bitcoin_price():
    return "Current Bitcoin Price: $52,300 (Mock Data)"

In [24]:
def route_prompt(user_input: str) -> str:
    if "bitcoin" in user_input.lower():
        return "tool"

    prompt = f"""
    Classify the following user query into exactly one of these categories:
    [technical, billing, general]

    Return ONLY the category name.

    Query: {user_input}
    """

    response = router_llm.invoke(prompt)
    return response.content.strip().lower()

In [25]:
def process_request(user_input: str) -> str:
    category = route_prompt(user_input)

    if category == "tool":
        return "[ROUTED → TOOL EXPERT]\n\n" + fetch_bitcoin_price()

    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]

    messages = [
        ("system", system_prompt),
        ("human", user_input)
    ]

    response = llm.invoke(messages)

    return f"[ROUTED → {category.upper()} EXPERT]\n\n{response.content}"

In [28]:
print(process_request("What is the current price of Bitcoin?"))

[ROUTED → TOOL EXPERT]

Current Bitcoin Price: $52,300 (Mock Data)
